In [1]:
import pandas as pd

# Ex 1

## 1

### Data

#### Import Data

In [37]:
char_cols = ["charID", "charName", "movieID", "mTitle", "sex", "credPos"]
characters = pd.read_csv(
    "./data/initial_data/movie_characters_metadata.tsv", 
    sep="\t",
    header=None,
    on_bad_lines = 'warn'
)
characters.columns=char_cols

Skipping line 6565: expected 6 fields, saw 13



In [38]:
characters.head()

,charID,charName,movieID,mTitle,sex,credPos
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [39]:
convers_cols = ["firstCharID", "secondCharID", "movieID", "utterances"]
conversations = pd.read_csv(
    "data/initial_data/movie_conversations.tsv", 
    sep="\t",
    header=None,
    on_bad_lines = 'warn'
)
conversations.columns = convers_cols

In [40]:
conversations["conv_list"] = conversations["utterances"].str.findall(r"[L]\d{1,}")

In [41]:
conversations.head()

,firstCharID,secondCharID,movieID,utterances,conv_list
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197'],"[L194, L195, L196, L197]"
1,u0,u2,m0,['L198' 'L199'],"[L198, L199]"
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203'],"[L200, L201, L202, L203]"
3,u0,u2,m0,['L204' 'L205' 'L206'],"[L204, L205, L206]"
4,u0,u2,m0,['L207' 'L208'],"[L207, L208]"


In [42]:
lines_cols = ["lineID", "characterID", "movieID", "charName", "aText"]
lines = pd.read_csv(
    "data/initial_data/movie_lines.tsv", 
    sep="\t",
    header=None,
    on_bad_lines = 'warn'
)
lines.columns=lines_cols

Skipping line 32288: expected 5 fields, saw 7
Skipping line 32351: expected 5 fields, saw 6
Skipping line 32390: expected 5 fields, saw 6
Skipping line 32583: expected 5 fields, saw 6
Skipping line 32585: expected 5 fields, saw 6
Skipping line 35684: expected 5 fields, saw 6
Skipping line 62132: expected 5 fields, saw 6
Skipping line 86637: expected 5 fields, saw 6
Skipping line 86722: expected 5 fields, saw 6
Skipping line 86914: expected 5 fields, saw 6
Skipping line 86960: expected 5 fields, saw 6
Skipping line 87010: expected 5 fields, saw 6
Skipping line 87011: expected 5 fields, saw 6
Skipping line 87086: expected 5 fields, saw 6
Skipping line 120607: expected 5 fields, saw 6
Skipping line 120719: expected 5 fields, saw 7
Skipping line 120739: expected 5 fields, saw 6
Skipping line 120783: expected 5 fields, saw 6
Skipping line 130284: expected 5 fields, saw 7
Skipping line 131048: expected 5 fields, saw 6

Skipping line 150955: expected 5 fields, saw 8
Skipping line 162777: expe

In [43]:
lines.sample(5)

,lineID,characterID,movieID,charName,aText
256785,L549915,u7883,m533,JACKIE,You think <u>I</u> didn't have some dirty word...
236515,L473112,u7311,m494,DEIRDRE,I'm a patriate.
46671,L295842,u1413,m94,BERT,Level with me Eddie. You ever play billiards b...
126403,L53511,u3801,m251,CHRIS,That makes you an expert I guess.
147939,L127405,u4440,m294,MISS DUNLOP,Hi Jody. What can I DO for you?


In [44]:
title_cols = ["movieID", "title", "year", "rating", "no.votes", "genres"]
titles = pd.read_csv(
    "data/initial_data/movie_titles_metadata.tsv", 
    sep="\t",
    header=None,
    on_bad_lines = 'warn'
)
titles.columns=title_cols

In [45]:
titles.head()

,movieID,title,year,rating,no.votes,genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


#### PrepareData

In [46]:
MAX_DIALOG_LEN = 10
def get_line(line_list: list[str]):
    df = lines.loc[lines['lineID'].isin(line_list)]
    chars = df["charName"].unique()
    dialog_line = df["aText"][::-1].values # reversed because pandas sort that way
    dialog_line = dialog_line[:MAX_DIALOG_LEN]
    return dialog_line, chars

In [47]:
dialog_line_chars = conversations["conv_list"].apply(lambda x: get_line(x))

In [48]:
df = conversations.copy()
df_merged = df.merge(titles, left_on="movieID", right_on="movieID", how="left")

In [87]:
df_final = df_merged[["title", "genres"]].copy()
df_final["lines_chars"] = dialog_line_chars

In [94]:
def try_eval(string):
    try:
        return pd.eval(string)
    except:
        pass
df_final['genres'] = df_final['genres'].str.replace(" ", ",").apply(try_eval);

In [95]:
df_final.head()

,title,genres,lines_chars
0,10 things i hate about you,"[comedy, romance]",([Can we make this quick? Roxanne Korrine and...
1,10 things i hate about you,"[comedy, romance]",([You're asking me out. That's so cute. What'...
2,10 things i hate about you,"[comedy, romance]",([No no it's my fault -- we didn't have a prop...
3,10 things i hate about you,"[comedy, romance]","([Why?, Unsolved mystery. She used to be real..."
4,10 things i hate about you,"[comedy, romance]",([Gosh if only we could find Kat a boyfriend.....


In [96]:
df_final.to_parquet("data/prepared_data/prepared.parquet")

## 2

### Ray Serve Realisation

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline 

/home/dmitry/.cache/pypoetry/virtualenvs/homework-2-yr0A23hy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_parquet("data/prepared_data/prepared.parquet")

In [3]:
# arr and string for testing 

arr = list(data["lines_chars"][0][0])
string = data["lines_chars"][0][0][0]
# frame = pd.DataFrame(data["lines_chars"][0][0])
# batch = [arr,arr]

In [4]:
import ray 
from ray import serve
from starlette.requests import Request

ray.init()
serve.start()

2023-06-17 11:02:57,521	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [5]:
import json
@serve.deployment(route_prefix="/")
class sentiment_pipeline:

    def __init__(self):
        self.pipeline = pipeline(
            model="finiteautomata/bertweet-base-sentiment-analysis"
        )

    def _unswer(self, text):
        return self.pipeline(text)[0]

    def __call__(self, request: Request) -> str:
        data = request.query_params["data"]
        text = json.loads(data)
        unswers = []
        if isinstance(text, list):
            for i, value in enumerate(text):
                unswer = self._unswer(text)
                unswer["sentence"] = i
                unswers.append(unswer)
        elif isinstance(text, str):
            unswer = self._unswer(text)
            unswer["sentence"] = 0
            unswers.append(unswer)
        else:
            raise TypeError(f"Unexpected type")
        
        return json.dumps(unswers)

sentiment_pipeline.deploy()

(ServeController pid=13293) INFO 2023-06-17 11:02:59,326 controller 13293 deployment_state.py:1298 - Deploying new version of deployment sentiment_pipeline.
(HTTPProxyActor pid=13339) INFO:     Started server process [13339]
(ServeController pid=13293) INFO 2023-06-17 11:02:59,377 controller 13293 deployment_state.py:1537 - Adding 1 replica to deployment sentiment_pipeline.


In [ ]:
import requests
import json

In [17]:
# for string
request = requests.get("http://127.0.0.1:8000/", params={"data": json.dumps(string)})
json.loads(request.text)

[{'label': 'NEG', 'score': 0.9783918261528015, 'sentence': 0}]

(ServeReplica:sentiment_pipeline pid=29110) INFO 2023-06-16 23:17:13,580 sentiment_pipeline sentiment_pipeline#xTlmBN CGUcfFvequ / replica.py:654 - __CALL__ OK 139.5ms


In [18]:
# for array
request = requests.get("http://127.0.0.1:8000/", params={"data": json.dumps(arr)})
json.loads(request.text)

[{'label': 'NEG', 'score': 0.9783918261528015, 'sentence': 0},
 {'label': 'NEG', 'score': 0.9783918261528015, 'sentence': 1},
 {'label': 'NEG', 'score': 0.9783918261528015, 'sentence': 2},
 {'label': 'NEG', 'score': 0.9783918261528015, 'sentence': 3}]

(ServeReplica:sentiment_pipeline pid=29110) INFO 2023-06-16 23:17:19,939 sentiment_pipeline sentiment_pipeline#xTlmBN jNbaRVOziT / replica.py:654 - __CALL__ OK 1980.2ms


In [23]:
arr = list(data["lines_chars"][100][0])
print(arr)

# for array
request = requests.get("http://127.0.0.1:8000/", params={"data": json.dumps(arr)})
print(json.loads(request.text))

['Yeah', 'What do you think?']
[{'label': 'POS', 'score': 0.7530627846717834, 'sentence': 0}, {'label': 'POS', 'score': 0.7530627846717834, 'sentence': 1}]


(ServeReplica:sentiment_pipeline pid=29110) INFO 2023-06-16 23:20:05,614 sentiment_pipeline sentiment_pipeline#xTlmBN dBmlOGZylM / replica.py:654 - __CALL__ OK 303.7ms


### FastAPI realisation

In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline

import requests
import json

/home/dmitry/.cache/pypoetry/virtualenvs/homework-2-yr0A23hy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_parquet("data/prepared_data/prepared.parquet")

In [3]:
arr = list(data["lines_chars"][0][0])
string = data["lines_chars"][0][0][0]

In [4]:
import ray 
from ray import serve
from starlette.requests import Request

ray.init()
serve.start()

2023-06-17 14:36:24,378	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [5]:
from pydantic import BaseModel
from typing import Union
from pydantic import constr, PositiveInt

str_shape = constr(min_length=1, max_length=200, strip_whitespace=True) 

class Schema(BaseModel):
    
    data: Union[str_shape, list[str_shape]]
    max_dialog_len: Union[PositiveInt, None] = None

In [6]:
import json
from fastapi import FastAPI


app = FastAPI()

@serve.deployment
@serve.ingress(app)
class sentiment_pipeline:

    def __init__(self):
        self.pipeline = pipeline(
            model="finiteautomata/bertweet-base-sentiment-analysis"
        )

    def _unswer(self, text):
        return self.pipeline(text)[0]

    @app.post(path="/estimate")
    def get_estimation(
        self,
        schema_json: Schema
    ) -> str:
        schema = Schema.parse_obj(schema_json)
        text = schema.data
        unswers = {}
        if isinstance(text, list):
            text = text[:schema.max_dialog_len]
            for i, value in enumerate(text):
                unswer = self._unswer(text)
                unswers[i] = unswer
        elif isinstance(text, str):
            unswer = self._unswer(text)
            unswers[0] = unswer
        
        return json.dumps(unswers)

sentiment_pipeline.deploy()

(HTTPProxyActor pid=31974) INFO:     Started server process [31974]
(ServeController pid=31929) INFO 2023-06-17 14:36:26,375 controller 31929 deployment_state.py:1298 - Deploying new version of deployment sentiment_pipeline.
(ServeController pid=31929) INFO 2023-06-17 14:36:26,393 controller 31929 deployment_state.py:1537 - Adding 1 replica to deployment sentiment_pipeline.
(ServeController pid=31929) WARNING 2023-06-17 14:36:56,450 controller 31929 deployment_state.py:1869 - Deployment sentiment_pipeline has 1 replicas that have taken more than 30s to initialize. This may be caused by a slow __init__ or reconfigure method.
(ServeReplica:sentiment_pipeline pid=32035) emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
(ServeReplica:sentiment_pipeline pid=32035) Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
(ServeReplic

In [11]:
# for string
request = requests.post(
    "http://127.0.0.1:8000/sentiment_pipeline/estimate", 
    json={"data": string}
)
print(request.json())

{"0": {"label": "NEG", "score": 0.9783918261528015}}


(ServeReplica:sentiment_pipeline pid=32035) INFO 2023-06-17 14:47:39,390 sentiment_pipeline sentiment_pipeline#zpOMVd JslEkCRVyw /sentiment_pipeline/estimate replica.py:654 - __CALL__ OK 207.5ms


In [8]:
# for array
print(len(arr)) # len 4

request = requests.post(
    "http://127.0.0.1:8000/sentiment_pipeline/estimate", 
    json={"data": arr, "max_dialog_len": 2}
)
print(request.json())

4


(ServeReplica:sentiment_pipeline pid=32035) INFO 2023-06-17 14:37:12,657 sentiment_pipeline sentiment_pipeline#zpOMVd gnoPYIjSeL /sentiment_pipeline/estimate replica.py:654 - __CALL__ OK 102.2ms


{"0": {"label": "NEG", "score": 0.9783918261528015}, "1": {"label": "NEG", "score": 0.9783918261528015}}


(ServeReplica:sentiment_pipeline pid=32035) INFO 2023-06-17 14:37:13,005 sentiment_pipeline sentiment_pipeline#zpOMVd iNymgjVWCz /sentiment_pipeline/estimate replica.py:654 - __CALL__ OK 332.6ms


### Interface

In [1]:
import pandas as pd
data = pd.read_parquet("data/prepared_data/prepared.parquet")

In [2]:
def make_dialog(line_chars):
    chars = line_chars[1]
    dialog = []
    if len(chars) == 2:
        it = 0
        for replic in line_chars[0]:
            if it % 2 == 0:
                dialog.append(f"${chars[0]} --{replic}")
            else:
                dialog.append(f"${chars[1]} --{replic}")
            it+=1
    else:
        for replic in line_chars[0]:
            dialog.append(f"${chars[0]} --{replic}")
    
    return "\n".join(dialog)

In [3]:
dialog_example1 = make_dialog(data.lines_chars[50])
dialog_example2 = make_dialog(data.lines_chars[100])
dialog_example3 = make_dialog(data.lines_chars[1232])

In [4]:
import ray 
from ray import serve
from starlette.requests import Request

ray.init()
serve.start(detached=True)

/home/dmitry/.cache/pypoetry/virtualenvs/homework-2-yr0A23hy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-17 19:30:53,362	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(HTTPProxyActor pid=30487) INFO:     Started server process [30487]


In [5]:
from pydantic import BaseModel
from typing import Union
from pydantic import constr, PositiveInt

str_shape = constr(min_length=1, max_length=1000, strip_whitespace=True) 

class Schema(BaseModel):
    
    data: Union[str_shape, list[str_shape]]
    max_dialog_len: Union[PositiveInt, None] = None

In [6]:
from transformers import pipeline
import json


class sentiment_pipeline:

    def __init__(self):
        self.pipeline = pipeline(
            model="finiteautomata/bertweet-base-sentiment-analysis"
        )

    def _unswer(self, text):
        return self.pipeline(text)[0]

    def get_estimation(
        self,
        schema_json: Schema
    ) -> str:
        schema = Schema.parse_obj(schema_json)
        text = schema.data
        unswers = {}
        if isinstance(text, list):
            text = text[:schema.max_dialog_len]
            for i, value in enumerate(text):
                unswer = self._unswer(value)
                unswers[i] = unswer
        elif isinstance(text, str):
            unswer = self._unswer(text)
            unswers[0] = unswer
        
        return json.dumps(unswers)

In [7]:
from fastapi import FastAPI

app = FastAPI()

@serve.deployment
@serve.ingress(app)
class api_santiment(sentiment_pipeline):

    def __init__(self):
        super().__init__()
    
    def _unswer(self, text: str | list):
        return super()._unswer(text)

    @app.post("/")
    def get_estimation(self, schema_json: Schema):
        return super().get_estimation(schema_json)

In [8]:
serve.run(api_santiment.bind(), name="santiment", route_prefix="/santiment")

The new client HTTP config differs from the existing one in the following fields: ['host', 'location']. The new HTTP config is ignored.
(ServeController pid=30440) INFO 2023-06-17 19:30:59,681 controller 30440 deployment_state.py:1298 - Deploying new version of deployment santiment_api_santiment.
(ServeController pid=30440) INFO 2023-06-17 19:30:59,724 controller 30440 deployment_state.py:1537 - Adding 1 replica to deployment santiment_api_santiment.
(ServeReplica:santiment_api_santiment pid=30598) emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
(ServeReplica:santiment_api_santiment pid=30598) Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
(ServeReplica:santiment_api_santiment pid=30598) pip install xformers.


RayServeSyncHandle(deployment='santiment_api_santiment')

In [9]:
import re


class gradio_santiment(sentiment_pipeline):

    def __init__(self):
        super().__init__()
    
    def _unswer(self, text: str | list):
        return super()._unswer(text)
    
    def get_estimation(self, text):
        corpus = re.split("\n", text)
        unswer = {}
        for sample in corpus:
            char = re.findall("^[$]\w{1,}", sample)[0]
            words = sample[re.search("(--)", sample).end():]
            schema_json = {"data": words}
            estimation = json.loads(super().get_estimation(schema_json))
            value = list(estimation.values())
            dict_value = value[0]
            if char in unswer.keys():
                unswer[char].append(dict_value["label"])
            else:
                unswer[char] = [dict_value["label"],]
        return unswer

In [51]:
import gradio as gr
from ray.serve.gradio_integrations import GradioServer
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter


class gradio():
    
    def __init__(self):
        self.pipeline = gradio_santiment()
    
    def _get_figures(self, text):

        uswer = json.loads(self.pipeline.get_estimation(text))
        len_keys = len(uswer.keys())
        fig = make_subplots(
            rows=1, 
            cols=len_keys,
            specs=[[{"type": "pie"} for i in range(len_keys)]]
        )

        for n, key in enumerate(uswer.keys(), 1):
            counter=Counter(uswer[key])
            labels = list(counter.keys())
            values = list(counter.values())
            fig.add_trace(
                go.Pie(labels=labels, values=values, title=key),
                row=1, col=n
            )
        
        return fig

    def __call__(self):
        with gr.Blocks() as demo:
            with gr.Row() as row1:
                with gr.Column() as col1:
                    input_textbox = gr.Textbox(label="Input")
                    button_run = gr.Button(label="run")
                    examples = gr.Examples(
                        [dialog_example1, dialog_example2, dialog_example3], 
                        [input_textbox]
                    )
                with gr.Column() as col2:
                    output_textbox = gr.Textbox(label="Output")
                    graph = gr.Plot()
                
            button_run.click(self.pipeline.get_estimation, input_textbox, output_textbox)
            button_run.click(self._get_figures, input_textbox, graph)
        
        return demo


def gradio_call():

    model = gradio()

    return model()

app = GradioServer.options(
    ray_actor_options={"num_cpus": 4},
).bind(gradio_call)

(ServeReplica:graph_GradioIngress pid=17183) INFO 2023-06-17 20:52:54,852 graph_GradioIngress graph_GradioIngress#qhidYt pFaFPPUGPC /graph/app_id graph replica.py:654 - __CALL__ OK 1.3ms


In [52]:
serve.run(app, name="graph", route_prefix="/graph");

The new client HTTP config differs from the existing one in the following fields: ['host', 'location']. The new HTTP config is ignored.
(ServeController pid=30440) INFO 2023-06-17 20:52:55,322 controller 30440 deployment_state.py:1298 - Deploying new version of deployment graph_GradioIngress.
(ServeController pid=30440) INFO 2023-06-17 20:52:55,331 controller 30440 deployment_state.py:1425 - Stopping 1 replicas of deployment 'graph_GradioIngress' with outdated versions.
(ServeController pid=30440) INFO 2023-06-17 20:52:57,507 controller 30440 deployment_state.py:1537 - Adding 1 replica to deployment graph_GradioIngress.
(ServeController pid=30440) WARNING 2023-06-17 20:53:27,541 controller 30440 deployment_state.py:1869 - Deployment graph_GradioIngress has 1 replicas that have taken more than 30s to initialize. This may be caused by a slow __init__ or reconfigure method.
(ServeReplica:graph_GradioIngress pid=17648) emoji is not installed, thus not converting emoticons or emojis into te

(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:53:43,506 graph_GradioIngress graph_GradioIngress#qtTmQg ijjQAClHVT /graph/info graph replica.py:654 - __CALL__ OK 1.9ms
(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:53:43,656 graph_GradioIngress graph_GradioIngress#qtTmQg LyfiLFkuNP /graph/theme.css graph replica.py:654 - __CALL__ OK 1.2ms
(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:53:43,718 graph_GradioIngress graph_GradioIngress#qtTmQg qvCiOMrZBX /graph/app_id graph replica.py:654 - __CALL__ OK 2.6ms
(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:53:44,121 graph_GradioIngress graph_GradioIngress#qtTmQg pcOHAevsIy /graph/app_id graph replica.py:654 - __CALL__ OK 2.5ms
(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:53:44,423 graph_GradioIngress graph_GradioIngress#qtTmQg XmDpITXAtc /graph/app_id graph replica.py:654 - __CALL__ OK 1.5ms
(ServeReplica:graph_GradioIngress pid=17648) INFO 2023-06-17 20:

## 3

### Prepare data

In [1]:
import pandas as pd
import numpy as np
import random
random.seed(10)

data = pd.read_parquet("data/prepared_data/prepared.parquet")


In [2]:

def data_prepare(data):

    def make_dialog(line_chars, genres, title):
        chars = line_chars[1]
        if genres is not None:
            genres = tuple(genres)
        else: 
            genres = ("None", )
        replics = {}
        try:
            if len(chars) == 2:
                it = 0
                for i, replic in enumerate(line_chars[0]):
                    if it % 2 == 0:
                        replics[i] = {
                            "char": chars[0], 
                            "replic": replic, 
                            "genre": genres, 
                            "title": title
                        }
                    else:
                        replics[i] = {
                            "char": chars[1], 
                            "replic": replic, 
                            "genre": genres, 
                            "title": title
                        }
                    it+=1
            else:
                for i, replic in enumerate(line_chars[0]):
                    replics[i] = {
                            "char": chars[0], 
                            "replic": replic, 
                            "genre": genres, 
                            "title": title
                        }
            
            return replics
        except:
            return {
                    "char": "aaa", 
                    "replic": "aaa", 
                    "genre": "aaa", 
                    "title": "aaa"
                }

    dialogs = []
    data_index = random.sample(list(data.index), 5000)
    for i in data_index:
        line_chars = data.loc[i, "lines_chars"]
        title = data.loc[i, "title"]
        genres = data.loc[i, "genres"]
        dialog = make_dialog(line_chars, genres, title)
        dialogs.append(dialog)

    return dialogs

In [3]:
dialogs = data_prepare(data)

In [4]:
dialog_frame = pd.DataFrame()
for dialog in dialogs:
    frame = pd.DataFrame(dialog).T
    dialog_frame = pd.concat([dialog_frame, frame], ignore_index=True)

In [5]:
dialog_frame.sample(5)

,char,replic,genre,title
8494,CATHERINE,When did you write it?,"(mystery, romance, thriller)",basic instinct
2495,VANESSA,I'm sure.,"(action, adventure, comedy, crime)",austin powers: international man of mystery
10811,MAX,Why?,"(comedy, crime, drama, thriller)",made
5360,MICHAELS,The Lymphatic System drains off excess fluid f...,"(adventure, sci-fi)",fantastic voyage
12110,FLORA,Yes and so do I but there are too few of us he...,"(drama, romance)",the piano


In [6]:
dialog_frame.to_parquet("data/prepared_data/dialogs.parquet")

#### emotion evaluation

In [1]:
import pandas as pd

data = pd.read_parquet("data/prepared_data/dialogs.parquet")

In [2]:
from transformers import pipeline

/home/dmitry/.cache/pypoetry/virtualenvs/homework-2-yr0A23hy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = pipeline(model="mrm8488/t5-base-finetuned-emotion")

In [4]:
def evaluate(string, model):
    try:
        evaluation = model(string)
        return evaluation[0]['generated_text']
    except:
        pass

In [5]:
df = data[:300].copy()

In [7]:
df2 = data.loc[data.title.isin(
    ['the haunting', 'black snake moan', 'jaws: the revenge','notting hill', 'the third man', 'amadeus']
)].copy()

In [6]:
df["emotion"] = df.replic.apply(
    lambda x: evaluate(x, model)
)

In [8]:
df2["emotion"] = df2.replic.apply(
    lambda x: evaluate(x, model)
)

In [9]:
df2.head()

,char,replic,genre,title,emotion
0,NELL,What did it smell like?,"[horror, thriller, mystery]",the haunting,fear
1,MARROW,It was very specific.,"[horror, thriller, mystery]",the haunting,joy
2,NELL,All right...,"[horror, thriller, mystery]",the haunting,joy
3,MARROW,In the bathroom in my mother's room the toilet...,"[horror, thriller, mystery]",the haunting,anger
4,NELL,So... smell... is... Smell is the sense that t...,"[horror, thriller, mystery]",the haunting,joy


In [10]:
df2.to_parquet("data/prepared_data/dialogs_emotions.parquet")

In [8]:
df.to_parquet("data/prepared_data/dialogs_emotions_sample.parquet")

In [11]:
data = pd.read_parquet("data/prepared_data/dialogs_emotions_sample.parquet")

data2 = pd.read_parquet("data/prepared_data/dialogs_emotions.parquet")

char_cols = ["charID", "charName", "movieID", "mTitle", "sex", "credPos"]
characters = pd.read_csv(
    "./data/initial_data/movie_characters_metadata.tsv", 
    sep="\t",
    header=None,
    on_bad_lines = 'warn'
)
characters.columns=char_cols

Skipping line 6565: expected 6 fields, saw 13



In [20]:
df = data.merge(characters, left_on=["char", "title"], right_on=["charName", "mTitle"], how="left")

In [12]:
df2 = data2.merge(characters, left_on=["char", "title"], right_on=["charName", "mTitle"], how="left") 

In [14]:
cols = ['char', 'replic', 'genre', 
        'title', 'emotion', 'sex']
df2 = df2[cols].copy()       


In [ ]:
df = df[cols].copy()

In [11]:
df.to_parquet("data/prepared_data/dialogs_sex_emotion.parquet")

In [16]:
df2.to_parquet("data/prepared_data/dialogs_sex_emotion_films.parquet")

### App realisation

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet("data/prepared_data/dialogs_sex_emotion.parquet")
data.genre = data.genre.apply(pd.eval)

In [3]:
data.sample(5)

,char,replic,genre,title,emotion,sex
52,PEGGY,I feel ridiculous. Maybe I should go home and ...,"[comedy, drama, fantasy, romance]",peggy sue got married,sadness,f
228,MARLA,You can't have both parasites. You can take b...,"[crime, drama, mystery, thriller]",fight club,fear,f
35,NANCY,Bring him out of what?,[horror],a nightmare on elm street,joy,f
157,JIMMY,I'm Bob.,[drama],magnolia,joy,m
33,PERFECT TOMMY,We're waiting for the Jet Car. Billy's bringi...,"[adventure, romance, comedy, sci-fi]",the adventures of buckaroo banzai across the 8...,joy,?


In [4]:
all_genres = []
[[all_genres.append(genre) for genre in genres] for genres in data.genre];
all_genres = np.unique(all_genres)

In [5]:
all_genres

array(['action', 'adventure', 'animation', 'biography', 'comedy', 'crime',
       'drama', 'family', 'fantasy', 'film-noir', 'history', 'horror',
       'music', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller',
       'war', 'western'], dtype='<U9')

In [6]:
import ray 
from ray import serve
from starlette.requests import Request

ray.init()
serve.start(detached=True)

/home/dmitry/.cache/pypoetry/virtualenvs/homework-2-yr0A23hy-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-18 03:10:35,753	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [7]:
import plotly.graph_objects as go
from collections import Counter
import gradio as gr


class gradio():

    def __init__(self, data, all_genres):
        self.data = data
        self.genres = all_genres
        self.choosen_genres = []

    def filter_df(self, choosen_genres):
        mask = [
            any(genre in choosen_genres for genre in genres) 
            for genres in data.genre.values
        ]
        df = self.data.loc[mask]

        return df.emotion.values

    def filter_map(self, choosen_genres):

        choosen_genres = self.parse(choosen_genres)
        
        emotions = Counter(
            self.filter_df(choosen_genres)
        )

        fig = go.Figure(
            go.Bar(
                x=list(emotions.keys()),
                y=list(emotions.values())
            )
        )

        return fig
    
    def parse(self, string: str):
        return string.split("|")
    
    def clear(self):
        return ""
        
    def get_app(self):
        with gr.Blocks() as demo:
            with gr.Row():
                output_text = gr.Textbox("")
            with gr.Row():
                for genre in self.genres:
                    with gr.Column(min_width=80):
                        button = gr.Button(genre, label=genre)
                        button.click(None, [], output_text, _js=f"(x) => x + '{genre}|'")
            with gr.Row():
                with gr.Column():
                    compute_button = gr.Button("compute", variant="primary")
                with gr.Column():
                    clear_button = gr.Button("clear genres", variant="stop")
            with gr.Row():
                plot = gr.Plot(min_width=90)
            
            compute_button.click(self.filter_map, output_text, plot)
            clear_button.click(self.clear, [], output_text)
            
        return demo
    

(HTTPProxyActor pid=14814) INFO:     Started server process [14814]


In [8]:
def get_app():

    app = gradio(data, all_genres)

    return app.get_app()

In [9]:
from ray.serve.gradio_integrations import GradioServer


app = GradioServer.options().bind(get_app)

In [10]:
serve.run(app, name="Frame stats", route_prefix="/frame_stats")

The new client HTTP config differs from the existing one in the following fields: ['location']. The new HTTP config is ignored.
(ServeController pid=14765) INFO 2023-06-18 03:10:42,014 controller 14765 deployment_state.py:1298 - Deploying new version of deployment Frame stats_GradioIngress.
(ServeController pid=14765) INFO 2023-06-18 03:10:42,017 controller 14765 deployment_state.py:1537 - Adding 1 replica to deployment Frame stats_GradioIngress.
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:46,773 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx HTifDMfyrr /frame_stats/ Frame stats replica.py:654 - __CALL__ OK 12.5ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:46,833 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx xUyZsQQjQL /frame_stats/assets/index-188ef5e8.js Frame stats replica.py:654 - __CALL__ OK 4.1ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:46,943 Frame stats_GradioIngr

RayServeSyncHandle(deployment='Frame stats_GradioIngress')

(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:47,104 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx ZtZkmeUHFB /frame_stats/assets/Blocks-658f67b5.js Frame stats replica.py:654 - __CALL__ OK 5.5ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:47,128 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx dpGFGkJyxd /frame_stats/assets/Button-04f93a38.js Frame stats replica.py:654 - __CALL__ OK 5.1ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:47,146 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx KGiILxInEy /frame_stats/assets/Button-ee8bd9eb.css Frame stats replica.py:654 - __CALL__ OK 3.9ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INFO 2023-06-18 03:10:47,166 Frame stats_GradioIngress Frame stats_GradioIngress#blwaEx OvZCbSZzns /frame_stats/assets/Blocks-e1f8fadf.css Frame stats replica.py:654 - __CALL__ OK 4.2ms
(ServeReplica:Frame stats_GradioIngress pid=14920) INF

# Ex 2